In [4]:
# import the necessary packages
from skimage.measure import compare_ssim
import imutils
import cv2
import matplotlib.pyplot as plt

### Detect difference via Webcam Stream

In [45]:
cap = cv2.VideoCapture(0)

# get current frame and previous frame
ret, current_frame = cap.read()
previous_frame = current_frame

# save default frame as origin frame for comparing later
originFrame = current_frame

while(cap.isOpened()):
    
    # get gray frames
    originFrame_gray = cv2.cvtColor(originFrame, cv2.COLOR_BGR2GRAY)
    current_frame_gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)
    
    # get difference between frames
    (score, diff) = compare_ssim(originFrame_gray, current_frame_gray, gaussian_weights=True, full=True)
    diff = (diff * 255).astype("uint8")
    # get diff score put text on screen (1 = same)
    cv2.putText(current_frame, "Score {0:.4f}".format(score), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

   
    if score< 0.97:
         # find frame_diff countour
        thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        for c in cnts:
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(current_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            #cv2.rectangle(current_frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
           
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        

    cv2.imshow('diff',diff)
    cv2.imshow('current_frame',previous_frame)
    
    previous_frame = current_frame.copy()
    ret, current_frame = cap.read()
    
    
    
cap.release()
cv2.destroyAllWindows()